In [56]:
import pandas as pd
import json as js
import numpy as np
# from unidecode import unidecode

In [69]:
def dash(df, num):
  for i in df.iloc[num]:
    if i == '-':
      return True
  return False

f23 = pd.read_csv("/content/drive/MyDrive/FIFA/futbin-23-big.csv", keep_default_na=False, na_values='-')
f22 = pd.read_csv("/content/drive/MyDrive/FIFA/futbin-22-big.csv", keep_default_na=False, na_values='-')
f21 = pd.read_csv("/content/drive/MyDrive/FIFA/futbin-21-big.csv", keep_default_na=False, na_values='-')
f20 = pd.read_csv("/content/drive/MyDrive/FIFA/futbin-20-big.csv", keep_default_na=False, na_values='-')
f19 = pd.read_csv("/content/drive/MyDrive/FIFA/futbin-19-big.csv", keep_default_na=False, na_values='-')
f18 = pd.read_csv("/content/drive/MyDrive/FIFA/futbin-18-big.csv", keep_default_na=False, na_values='-')
f17 = pd.read_csv("/content/drive/MyDrive/FIFA/futbin-17-big.csv", keep_default_na=False, na_values='-')


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (57) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [70]:
## Getting rid of the datapoints with a bunch of missing values, because those exist for some reason

for i,j in zip([f23,f22,f21,f20,f19,f18,f17],[23,22,21,20,19,18,17]):
    keep = []
    for k in range(i.shape[0]):
        if i.shape[1] - i.iloc[k].count() <= 3:
            keep.append(k)
    exec(f"f{j} = f{j}.iloc[{keep}].reset_index(drop=True)")

## Getting rid of the columns with a bunch of missing values, because those exist for some reason
for i,j in zip([f23,f22,f21,f20,f19,f18,f17],[23,22,21,20,19,18,17]):
    keep = []
    for k in range(i.shape[1]):
        if i.shape[0] - i.iloc[:,k].count() <= 1000:
            keep.append(k)
    exec(f"f{j} = f{j}.iloc[:,{keep}].reset_index(drop=True)")

In [71]:
## Getting rid of all of the goalkeepers (not relevant to ML study)

for i,j in zip([f23,f22,f21,f20,f19,f18,f17],[23,22,21,20,19,18,17]):
  keep = []
  for k in range(i.shape[0]):
    if i.at[k,"position"] != "GK":
      keep.append(k)
  exec(f"f{j} = f{j}.iloc[keep].reset_index(drop=True)")

In [72]:
# Adjusting Datatypes
# turning workrates into numerical data
# Collapsing Uncommonly Used Positions
wr_map = {
    "Low": 0,
    "Med": 1,
    "High": 2
}

pos_map = {
    "RF": "RW",
    "LF": "LW",
    "RWB": "RB",
    "LWB": "LB",
    "CF": "ST"
}

for i,j in zip([f23, f22,f21,f20,f19,f18,f17],[23,22,21,20,19,18,17]):
  i["Height"] = [np.NAN if type(k) != str or "cm" not in k else int((k.split("cm")[0])) for k in i["Height"]]
  i["Age"] = [np.NAN if type(k) != str or len(k) == 0 else int(k.split(" ")[0]) for k in i["Age"]]
  i["Foot"] = [(0 if k == "Right" else 1) for k in i["Foot"]]
  i["xbox_gp"] = [0 if type(k) != str else int("".join(i.xbox_gp.iloc[k].split(","))) for k in range(i.shape[0])]
  i["ps4_gp"] = [0 if type(k) != str else int("".join(i.ps4_gp.iloc[k].split(","))) for k in range(i.shape[0])]
  i["Att. WR"] = [np.NAN if k not in wr_map else wr_map[k] for k in i["Att. WR"]]
  i["Def. WR"] = [np.NAN if k not in wr_map else wr_map[k] for k in i["Def. WR"]]
  i["position"] = [pos_map[k] if k in pos_map else k for k in i["position"]]


In [73]:
## binarizing traits - we can't embed because each player doesn't have the same amount
## We can't use pd.get_dummies to binarize because the traits are list objects
tr_map = {
  'Speed Dribbler (CPU AI)': 'Speed Dribbler',
  'Chip Shot (CPU AI Only)': "Chip Shot",
  'Chip Shot (CPU AI)': "Chip Shot",
  'Dives Into Tackles (CPU AI Only)': "Dives Into Tackles",
  'Dives Into Tackles (CPU AI)': "Dives Into Tackles",
  'Crosser - Early Crosser': "Early Crosser",
  'Giant Throw-in': "Long Throw",
  'Giant Throw In': "Long Throw",
  'Long Throw-In': "Long Throw",
  'Long Passer (CPU AI Only)': "Long Passer",
 'Long Passer (CPU AI)': "Long Passer",
 'Long Shot Taker (CPU AI Only)': 'Long Shot Taker',
 'Long Shot Taker (CPU AI)': 'Long Shot Taker',
 'Playmaker (CPU AI Only)': 'Playmaker',
 'Playmaker (CPU AI)': 'Playmaker',
 'Shooting - Chip Shot': "Chip Shot",
 'Shooting - Finesse Shot': 'Finesse Shot',
 'Shooting - Long Shot Taker': 'Long Shot Taker',
 'Shooting - Outside Foot Shot': 'Outside Foot Shot',
 'Takes Finesse Style Free Kicks': 'Takes Finesse Style Free Kicks',
 'Takes Powerful Driven Free Kicks': 'Power Free-Kick',
 'Technical Dribbler (CPU AI Only)': 'Technical Dribbler',
 'Technical Dribbler (CPU AI)': 'Technical Dribbler'
}

for i,j in zip([f23,f22,f21,f20,f19,f18,f17],[23,22,21,20,19,18,17]):
  for k in range(i.shape[0]):
    if type(i.traits.iloc[k]) == str:
        lst = i.traits.iloc[k][2:-2].split("', '")
        i.traits.iloc[k]  = [l for l in lst if len(l) > 2]
        i.traits.iloc[k] = [tr_map[l] if l in tr_map else l for l in lst]
  
for i,j in zip([f23,f22,f21,f20,f19,f18,f17],[23,22,21,20,19,18,17]):
  for k in range(i.shape[0]):
    if type(i.traits.iloc[k]) == list:
        for l in i.traits.iloc[k]:
            if l not in i.columns:
                i[l] = [(1 if (type(m) == str and l in m) else 0) for m in i.traits]
  exec(f"f{j} = f{j}.drop(columns=['traits'])")

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-73-eb488b02ac15>:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  i[l] = [(1 if (type(m) == str and l in m) else 0) for m in i.traits]


In [74]:
"""CREDITS TO 
https://github.com/ActiveState/code/tree/3b27230f418b714bc9a0f897cb8ea189c3515e99/recipes/Python/576696_OrderedSet_with_Weakrefs
FOR ORDERED SET CODE
"""


import collections
from weakref import proxy

class Link(object):
    __slots__ = 'prev', 'next', 'key', '__weakref__'

class OrderedSet(collections.MutableSet):
    'Set the remembers the order elements were added'
    # Big-O running times for all methods are the same as for regular sets.
    # The internal self.__map dictionary maps keys to links in a doubly linked list.
    # The circular doubly linked list starts and ends with a sentinel element.
    # The sentinel element never gets deleted (this simplifies the algorithm).
    # The prev/next links are weakref proxies (to prevent circular references).
    # Individual links are kept alive by the hard reference in self.__map.
    # Those hard references disappear when a key is deleted from an OrderedSet.

    def __init__(self, iterable=None):
        self.__root = root = Link()         # sentinel node for doubly linked list
        root.prev = root.next = root
        self.__map = {}                     # key --> link
        if iterable is not None:
            self |= iterable

    def __len__(self):
        return len(self.__map)

    def __contains__(self, key):
        return key in self.__map

    def add(self, key):
        # Store new key in a new link at the end of the linked list
        if key not in self.__map:
            self.__map[key] = link = Link()            
            root = self.__root
            last = root.prev
            link.prev, link.next, link.key = last, root, key
            last.next = root.prev = proxy(link)

    def discard(self, key):
        # Remove an existing item using self.__map to find the link which is
        # then removed by updating the links in the predecessor and successors.        
        if key in self.__map:        
            link = self.__map.pop(key)
            link.prev.next = link.next
            link.next.prev = link.prev

    def __iter__(self):
        # Traverse the linked list in order.
        root = self.__root
        curr = root.next
        while curr is not root:
            yield curr.key
            curr = curr.next

    def __reversed__(self):
        # Traverse the linked list in reverse order.
        root = self.__root
        curr = root.prev
        while curr is not root:
            yield curr.key
            curr = curr.prev

    def pop(self, last=True):
        if not self:
            raise KeyError('set is empty')
        key = next(reversed(self)) if last else next(iter(self))
        self.discard(key)
        return key

    def __repr__(self):
        if not self:
            return '%s()' % (self.__class__.__name__,)
        return '%s(%r)' % (self.__class__.__name__, list(self))

    def __eq__(self, other):
        if isinstance(other, OrderedSet):
            return len(self) == len(other) and list(self) == list(other)
        return not self.isdisjoint(other)

In [75]:
## Removing Features that don't appear in all FIFA's
cols = list(OrderedSet(f23.columns) & OrderedSet(f22.columns) & OrderedSet(f21.columns) & OrderedSet(f20.columns) & OrderedSet(f19.columns) & OrderedSet(f18.columns) & OrderedSet(f17.columns))
for i in [23,22,21,20,19,18,17]:
  exec(f"f{i} = f{i}[{cols}]")

In [76]:
## Getting Rid of Junk Columns
for i,j in zip([f23, f22,f21,f20,f19,f18,f17],[23,22,21,20,19,18,17]):
  drops = []
  for k in range(len(i.columns)):
    if type(i.columns[k]) != str or i.columns[k] == '':
      drops.append(k)
  exec(f"f{j} = f{j}.iloc[:,[l for l in range(len(i.columns)) if l not in drops]].reset_index()")

In [77]:
cols=['ID', 'Origin', 'Added on', 'Revision', 'Intl. Rep', 'Club', 'League', 'index', 'ps4_gp', 'xbox_gp']
for i in [23,22,21,20,19,18,17]:
  exec(f"f{i} = f{i}.drop(columns={cols})")

In [79]:
for i,j in zip([f23, f22,f21,f20,f19,f18,f17],[23,22,21,20,19,18,17]):
    keep = []
    for k in range(i.shape[0]):
        try:
            int(i.at[k,"strength"])
            int(i.at[k,"aggression"])
            keep.append(k)
        except:
            pass
    exec(f"f{j} = f{j}.iloc[{keep}].reset_index(drop=True)")

In [81]:
f23.to_csv("/content/drive/MyDrive/FIFA/FUTBIN Usable/futbin-23-usable.csv", index=False)
f22.to_csv("/content/drive/MyDrive/FIFA/FUTBIN Usable/futbin-22-usable.csv", index=False)
f21.to_csv("/content/drive/MyDrive/FIFA/FUTBIN Usable/futbin-21-usable.csv", index=False)
f20.to_csv("/content/drive/MyDrive/FIFA/FUTBIN Usable/futbin-20-usable.csv", index=False)
f19.to_csv("/content/drive/MyDrive/FIFA/FUTBIN Usable/futbin-19-usable.csv", index=False)
f18.to_csv("/content/drive/MyDrive/FIFA/FUTBIN Usable/futbin-18-usable.csv", index=False)
f17.to_csv("/content/drive/MyDrive/FIFA/FUTBIN Usable/futbin-17-usable.csv", index=False)

In [87]:
for i in f22.columns:
    if not any(f22[i].isna()):
        print(i)

Name
Nation
Skills
Weak Foot
Foot
Weight
Att. WR
Def. WR
Age
Club ID
League ID
rat
position
pace
acceleration
sprintspeed
shooting
positioning
finishing
shotpower
longshotsaccuracy
volleys
penalties
passing
vision
crossing
freekickaccuracy
shortpassing
longpassing
curve
dribblingp
agility
balance
reactions
ballcontrol
dribbling
composure
defending
interceptions
headingaccuracy
marking
standingtackle
slidingtackle
heading
jumping
stamina
strength
aggression
pop
Power Free-Kick
Flair
Long Shot Taker
Finesse Shot
Technical Dribbler
Long Throw
Injury Prone
Chip Shot
Dives Into Tackles
Leadership
Power Header
Playmaker
Early Crosser
